In [1]:
import numpy as np
import time
import cssm

# Test parameters
v = np.array([0.5], dtype=np.float32)
a = np.array([1.0], dtype=np.float32)
z = np.array([0.5], dtype=np.float32)
t = np.array([0.3], dtype=np.float32)
deadline = np.array([3.0], dtype=np.float32)
s = np.array([1.0], dtype=np.float32)

n_samples = 50000
n_trials = 1

print("=" * 70)
print("COMPARING DDM vs DDM_NUMPY")
print("=" * 70)

# Test 1: Correctness (same seed should give similar distributions)
print("\n1. STATISTICAL CORRECTNESS TEST")
print("-" * 70)

results_old = cssm.ddm(v, a, z, t, deadline, s,
                       n_samples=n_samples, n_trials=n_trials,
                       random_state=42)

results_new = cssm.ddm_numpy(v, a, z, t, deadline, s,
                              n_samples=n_samples, n_trials=n_trials,
                              random_state=42)

print(f"Old DDM:")
print(f"  Mean RT:    {np.mean(results_old['rts'][results_old['rts'] > 0]):.4f}")
print(f"  Std RT:     {np.std(results_old['rts'][results_old['rts'] > 0]):.4f}")
print(f"  P(choice=1): {np.mean(results_old['choices'] == 1):.4f}")

print(f"\nNew DDM (NumPy):")
print(f"  Mean RT:    {np.mean(results_new['rts'][results_new['rts'] > 0]):.4f}")
print(f"  Std RT:     {np.std(results_new['rts'][results_new['rts'] > 0]):.4f}")
print(f"  P(choice=1): {np.mean(results_new['choices'] == 1):.4f}")

# Distributions should be similar (not identical due to different RNGs)
from scipy import stats
ks_stat, p_value = stats.ks_2samp(
    results_old['rts'][results_old['rts'] > 0].flatten(),
    results_new['rts'][results_new['rts'] > 0].flatten()
)
print(f"\nKolmogorov-Smirnov test:")
print(f"  KS Statistic: {ks_stat:.6f}")
print(f"  P-value:      {p_value:.6f}")
print(f"  Result: {'✅ Similar distributions' if p_value > 0.01 else '❌ Different distributions'}")

# Test 2: Performance comparison
print("\n2. PERFORMANCE TEST")
print("-" * 70)

n_runs = 5

times_old = []
times_new = []

for i in range(n_runs):
    # Old version
    start = time.perf_counter()
    _ = cssm.ddm(v, a, z, t, deadline, s,
                 n_samples=n_samples, n_trials=n_trials,
                 random_state=42+i)
    times_old.append(time.perf_counter() - start)
    
    # New version
    start = time.perf_counter()
    _ = cssm.ddm_numpy(v, a, z, t, deadline, s,
                       n_samples=n_samples, n_trials=n_trials,
                       random_state=42+i)
    times_new.append(time.perf_counter() - start)

mean_old = np.mean(times_old)
mean_new = np.mean(times_new)
speedup = mean_old / mean_new

print(f"Old DDM (Marsaglia):  {mean_old:.4f}s ± {np.std(times_old):.4f}s")
print(f"New DDM (NumPy):      {mean_new:.4f}s ± {np.std(times_new):.4f}s")
print(f"\n🚀 Speedup: {speedup:.2f}x")

if speedup > 1.5:
    print("✅ Significant speedup achieved!")
elif speedup > 1.1:
    print("⚠️  Modest speedup (expected ~1.7-2x)")
else:
    print("❌ No speedup - something might be wrong")

# Test 3: Larger scale test
print("\n3. LARGE SCALE TEST")
print("-" * 70)

n_samples_large = 100000

start = time.perf_counter()
_ = cssm.ddm(v, a, z, t, deadline, s,
             n_samples=n_samples_large, n_trials=1,
             random_state=42)
time_old_large = time.perf_counter() - start

start = time.perf_counter()
_ = cssm.ddm_numpy(v, a, z, t, deadline, s,
                   n_samples=n_samples_large, n_trials=1,
                   random_state=42)
time_new_large = time.perf_counter() - start

print(f"Simulating {n_samples_large:,} samples:")
print(f"  Old DDM:  {time_old_large:.4f}s")
print(f"  New DDM:  {time_new_large:.4f}s")
print(f"  Speedup:  {time_old_large/time_new_large:.2f}x")

print("\n" + "=" * 70)
print("TEST COMPLETE")
print("=" * 70)

COMPARING DDM vs DDM_NUMPY

1. STATISTICAL CORRECTNESS TEST
----------------------------------------------------------------------
Old DDM:
  Mean RT:    0.5630
  Std RT:     0.2122
  P(choice=1): 0.6296

New DDM (NumPy):
  Mean RT:    0.5641
  Std RT:     0.2137
  P(choice=1): 0.6276

Kolmogorov-Smirnov test:
  KS Statistic: 0.005260
  P-value:      0.491920
  Result: ✅ Similar distributions

2. PERFORMANCE TEST
----------------------------------------------------------------------
Old DDM (Marsaglia):  0.1967s ± 0.0021s
New DDM (NumPy):      0.0928s ± 0.0012s

🚀 Speedup: 2.12x
✅ Significant speedup achieved!

3. LARGE SCALE TEST
----------------------------------------------------------------------
Simulating 100,000 samples:
  Old DDM:  0.4030s
  New DDM:  0.1852s
  Speedup:  2.18x

TEST COMPLETE
